# Task 1 (Detecting pathological gamblers)
### Using contextualized language models

In [1]:
P_FILE = "../posts.csv"
TRAIN_TOKEN="../train_df.csv"
TEST_TOKEN="../test_df.csv"
GENERAL_MODELS="../Models"
ROLLING_WINDOW_SIZE=10
LM_MODEL="all-mpnet-base-v2"
CONVERTED=True
MODEL_PATH =f"{GENERAL_MODELS}/LM/NN_win_{ROLLING_WINDOW_SIZE}" 



In [2]:
from pathlib import Path
Path(MODEL_PATH).mkdir(parents=True, exist_ok=True)

### opening resulting dataset with pandas

In [3]:
import pandas as pd
import numpy as np
import os
seed=23
np.random.seed(seed)

train_df = pd.read_csv(TRAIN_TOKEN, sep='\t')
test_df = pd.read_csv(TEST_TOKEN, sep='\t')
train_df

,User,Post_Nr,Raw,Stemmed,Lemmatized,Label
0,3450,0,"sports betting number k in debt, feeling very ...","sport bet number k in debt , feel veri depress...","sports betting number k in debt , feeling very...",1
1,3450,1,finally accepted that you cannot win gambling ...,final accept that you can not win gambl relaps...,finally accepted that you can not win gambling...,1
2,3450,2,blocking software betfilter has anybody used t...,"block softwar betfilt has anybodi use this , w...",blocking software betfilter has anybody used t...,1
3,3450,3,prone to relapse when in debt? i find that whe...,prone to relaps when in debt ? i find that whe...,prone to relapse when in debt ? i find that wh...,1
4,3450,4,down to my last number on credit card i am num...,down to my last number on credit card i am num...,down to my last number on credit card i am num...,1
...,...,...,...,...,...,...
97853,162,670,you sick fuck,you sick fuck,you sick fuck,0
97854,162,671,reeeeeeeeeeeeeeeeeeeeeeeeeee,reeeeeeeeeeeeeeeeeeeeeeeeeee,reeeeeeeeeeeeeeeeeeeeeeeeeee,0
97855,162,672,this is so clearly satire! i really do not wan...,this is so clear satir ! i realli do not want ...,this is so clearly satire ! i really do not wa...,0
97856,162,673,he cannot write a story in number minutes,he can not write a stori in number minut,he can not write a story in number minutes,0


In [4]:
def rolling_window(df, window_size,stride, field):
    res_map={}
    for user in df['User'].unique():
        user_df = df[df['User']==user]
        res_map[user]=(user_df['Label'].values[0],{})
        posts = user_df[field].values
        iteration=0
        for i in range(0,len(posts),stride):
            res_map[user][1][iteration]=' '.join((posts[i:i+window_size]))
            iteration+=1
    result_df = pd.DataFrame([(k,k1,v1,v[0]) for k,v in res_map.items() for k1,v1 in v[1].items()], columns = ['User','Window_id','Text','Label'])
    
    return result_df

In [5]:
train_df = rolling_window(train_df,ROLLING_WINDOW_SIZE,1,'Raw')
test_df = rolling_window(test_df,ROLLING_WINDOW_SIZE,1,'Raw')

train_df

,User,Window_id,Text,Label
0,3450,0,"sports betting number k in debt, feeling very ...",1
1,3450,1,finally accepted that you cannot win gambling ...,1
2,3450,2,blocking software betfilter has anybody used t...,1
3,3450,3,prone to relapse when in debt? i find that whe...,1
4,3450,4,down to my last number on credit card i am num...,1
...,...,...,...,...
97853,162,670,you sick fuck reeeeeeeeeeeeeeeeeeeeeeeeeee thi...,0
97854,162,671,reeeeeeeeeeeeeeeeeeeeeeeeeee this is so clearl...,0
97855,162,672,this is so clearly satire! i really do not wan...,0
97856,162,673,he cannot write a story in number minutes pay ...,0


In [6]:
import pickle
#Store sentences & embeddings on disc
def save_embeddings(filepath, embeddings):
    with open(filepath, "wb") as fOut:
        pickle.dump({ 'embeddings': embeddings}, fOut, protocol=pickle.HIGHEST_PROTOCOL)

#Load sentences & embeddings from disc
def load_embeddings(filepath):
    with open(filepath, "rb") as fIn:
        stored_data = pickle.load(fIn)
        stored_embeddings = stored_data['embeddings']
    return stored_embeddings   

In [7]:
if not CONVERTED:
    from sentence_transformers import SentenceTransformer
    model = SentenceTransformer(LM_MODEL)

    model.max_seq_length = 512
    train_sentence_embeddings = model.encode(train_df['Text'],show_progress_bar=True,\
                output_value='sentence_embedding', batch_size=64,convert_to_numpy=True)

    val_sentence_embeddings = model.encode(test_df['Text'],show_progress_bar=True,\
                output_value='sentence_embedding', batch_size=64,convert_to_numpy=True)



In [8]:
#if not CONVERTED:
#    save_embeddings(f"./NN_train_sentence_embeddings_{LM_MODEL}_{ROLLING_WINDOW_SIZE}.pkl",train_sentence_embeddings)
#    save_embeddings(f"./NN_val_sentence_embeddings_{LM_MODEL}_{ROLLING_WINDOW_SIZE}.pkl",val_sentence_embeddings)

train_sentence_embeddings = load_embeddings(f"./train_sentence_embeddings_{LM_MODEL}_{ROLLING_WINDOW_SIZE}.pkl")
val_sentence_embeddings = load_embeddings(f"./val_sentence_embeddings_{LM_MODEL}_{ROLLING_WINDOW_SIZE}.pkl")



In [9]:

test_df['Vector'] = pd.DataFrame(data=val_sentence_embeddings).values.tolist()
train_df['Vector'] = pd.DataFrame(data=train_sentence_embeddings).values.tolist()


In [10]:
train_df.to_csv("lm_train_df_10.csv",index=False)
test_df.to_csv("lm_val_df_10.csv",index=False)


In [11]:
train_df

,User,Window_id,Text,Label,Vector
0,3450,0,"sports betting number k in debt, feeling very ...",1,"[0.001481963787227869, 0.08722537010908127, -0..."
1,3450,1,finally accepted that you cannot win gambling ...,1,"[0.0011301715858280659, 0.09679407626390457, -..."
2,3450,2,blocking software betfilter has anybody used t...,1,"[0.0012601062189787626, 0.08274412155151367, -..."
3,3450,3,prone to relapse when in debt? i find that whe...,1,"[0.018009161576628685, 0.05251404270529747, -0..."
4,3450,4,down to my last number on credit card i am num...,1,"[-0.006508147809654474, 0.08554978668689728, -..."
...,...,...,...,...,...
97853,162,670,you sick fuck reeeeeeeeeeeeeeeeeeeeeeeeeee thi...,0,"[0.04130152612924576, 0.0463968887925148, -0.0..."
97854,162,671,reeeeeeeeeeeeeeeeeeeeeeeeeee this is so clearl...,0,"[0.044361602514982224, 0.03786471486091614, -0..."
97855,162,672,this is so clearly satire! i really do not wan...,0,"[0.04709811136126518, 0.04260839894413948, -0...."
97856,162,673,he cannot write a story in number minutes pay ...,0,"[-0.017870843410491943, 0.004002168774604797, ..."


## Model part

In [12]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

torch.manual_seed(seed)


class WritingWindowDataset(Dataset):
    def __init__(self, vectors, labels):

        self.labels = [label for label in labels]
        self.vectors = [vector for vector in vectors]
    def classes(self):
        return self.labels

    def __len__(self):
        return len(self.labels)

    def get_batch_labels(self, idx):
        # Fetch a batch of labels
        return np.array(self.labels[idx])

    def get_batch_vectors(self, idx):
        # Fetch a batch of inputs
        return self.vectors[idx]

    def __getitem__(self, idx):

        batch_vectors = self.get_batch_vectors(idx)
        batch_y = self.get_batch_labels(idx)
        

        return batch_vectors, batch_y

In [13]:
full_ds = WritingWindowDataset(np.concatenate((train_sentence_embeddings, val_sentence_embeddings), axis=0), pd.concat([train_df['Label'], test_df['Label']]))

In [14]:
class LmNeuralNetwork(nn.Module):
    def __init__(self, trial):
        super(LmNeuralNetwork, self).__init__()
        
        self.layers=[]
        n_layers = trial.suggest_int("n_layers", 1, 3)

        in_features = 768
        for i in range(n_layers):
            out_features = trial.suggest_int("n_units_l{}".format(i), 4, min(in_features,128))
            self.layers.append(nn.Linear(in_features, out_features))
            self.layers.append(nn.ReLU())
            p = trial.suggest_float("dropout_l{}".format(i), 0.2, 0.5, step=0.05)
            self.layers.append(nn.Dropout(p))

            in_features = out_features
        self.layers.append(nn.Linear(in_features, 1))
        self.cls_layers = torch.nn.ModuleList(self.layers)
    def forward(self, x):

        for layer in self.cls_layers:
            x = layer(x)
        return torch.sigmoid(x)


In [15]:
from sklearn.model_selection import StratifiedKFold
n_folds=5
skf = StratifiedKFold(n_splits=n_folds)
def objective(trial):
    
    user_label_df =train_df.drop_duplicates('User')
    users = user_label_df['User'].to_numpy()
    labels = user_label_df['Label'].to_numpy()

    params = {'learning_rate': trial.suggest_loguniform('learning_rate', 1e-5, 1e-2)}
    score=[]
    for fold, (user_train_ids, user_val_ids) in enumerate(skf.split(users,labels)):
        model= LmNeuralNetwork(trial)
        train_users = [users[f] for f in user_train_ids]
        val_users = [users[f] for f in user_val_ids]
        train_ids = train_df.index[train_df['User'].isin(train_users)].to_list()
        val_ids = train_df.index[train_df['User'].isin(val_users)].to_list()

        train_data = torch.utils.data.Subset(full_ds, train_ids)
        val_data = torch.utils.data.Subset(full_ds, val_ids)

        trainloader = torch.utils.data.DataLoader(
                        train_data, 
                        batch_size=32, 
                        shuffle=True)
        valloader = torch.utils.data.DataLoader(
                        val_data,
                        batch_size=32,
                        shuffle=True)
        print(f"Fold {fold}")
        res = train_eval(trial,model,params,10,trainloader,valloader)
        score.append(res)
    return np.mean(score)
       

In [16]:
from torch.optim import AdamW
from tqdm import tqdm
from sklearn.metrics import f1_score
BATCH_SIZE = 32

def train_eval(trial,model, params, epochs, train_dataloader, val_dataloader):

   
    model = model.cuda()
    optimizer = AdamW(model.parameters(), lr=params['learning_rate'])#, lr= learning_rate)

    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")

    criterion = nn.BCELoss()
    
            

    for epoch_num in range(epochs):
       
        total_loss_train = 0
        train_preds = np.array([])
        train_targets = np.array([])
        model.train()
        for train_input, train_label in tqdm(train_dataloader):

            train_label = train_label.to(device)
            train_label = train_label.float()
            train_label = train_label.unsqueeze(1)
            features = train_input.to(device)
            

            output = model(features)
            batch_loss = criterion(output, train_label)
            total_loss_train += float(batch_loss.item())
            
            train_preds = np.concatenate((train_preds,torch.round(output).detach().cpu().numpy().flatten()),axis=0)
            train_targets = np.concatenate((train_targets,train_label.detach().cpu().numpy().flatten()),axis=0)

            
            model.zero_grad()
            batch_loss.backward()
            optimizer.step()
        train_loss=total_loss_train/len(train_preds)
        train_f1 = f1_score(train_targets,train_preds)

        total_loss_val = 0
        val_preds = np.array([])
        val_targets = np.array([])
        model.eval()
        with torch.no_grad():

            for val_input, val_label in val_dataloader:

                val_label = val_label.to(device)
                val_label = val_label.float()
                val_label = val_label.unsqueeze(1)

                features = val_input.to(device)

                output = model(features)

                batch_loss = criterion(output, val_label)
                total_loss_val += float(batch_loss.item())
                val_preds = np.concatenate((val_preds,torch.round(output).detach().cpu().numpy().flatten()),axis=0)
                val_targets = np.concatenate((val_targets,val_label.detach().cpu().numpy().flatten()),axis=0)

                
        val_f1 = f1_score(val_targets,val_preds)
        val_loss=total_loss_val/len(val_preds)
        
        
        print(
                f'Epochs: {epoch_num + 1} | Train Loss: {train_loss: .4f} \
                | Train F1: {train_f1: .4f} \
                | Val Loss: {val_loss: .4f} \
                | Val F1: {val_f1: .4f}')


    return val_f1


In [17]:
import optuna
import joblib
study = optuna.create_study(study_name=f"LM_NN_Tuning_{ROLLING_WINDOW_SIZE}_{LM_MODEL}",direction="maximize", sampler=optuna.samplers.TPESampler(), pruner=optuna.pruners.NopPruner())
#study = joblib.load(f"hyperparameterstudy{ROLLING_WINDOW_SIZE}.pkl")



[I 2022-06-16 15:50:23,892] A new study created in memory with name: LM_NN_Tuning_10_all-mpnet-base-v2


In [18]:
study.optimize(objective, n_trials=20, timeout=(60*60*3), gc_after_trial=True)
joblib.dump(study, f"NN_hyperparameterstudy{ROLLING_WINDOW_SIZE}.pkl")


Fold 0


100%|██████████| 2399/2399 [00:14<00:00, 168.64it/s]


Epochs: 1 | Train Loss:  0.0153                 | Train F1:  0.8119                 | Val Loss:  0.0134                 | Val F1:  0.7718


100%|██████████| 2399/2399 [00:06<00:00, 354.25it/s]


Epochs: 2 | Train Loss:  0.0102                 | Train F1:  0.8525                 | Val Loss:  0.0133                 | Val F1:  0.7680


...

100%|██████████| 2303/2303 [00:07<00:00, 327.07it/s]


Epochs: 9 | Train Loss:  0.0139                 | Train F1:  0.6517                 | Val Loss:  0.0156                 | Val F1:  0.7254


100%|██████████| 2303/2303 [00:07<00:00, 328.56it/s]
[I 2022-06-16 18:56:01,109] Trial 17 finished with value: 0.7906359712557505 and parameters: {'learning_rate': 0.00011256224938528428, 'n_layers': 2, 'n_units_l0': 4, 'dropout_l0': 0.45, 'n_units_l1': 4, 'dropout_l1': 0.30000000000000004}. Best is trial 15 with value: 0.807087138352161.


Epochs: 10 | Train Loss:  0.0137                 | Train F1:  0.6546                 | Val Loss:  0.0156                 | Val F1:  0.7243


['NN_hyperparameterstudy10.pkl']

In [23]:
study.optimize(objective, n_trials=7, timeout=(60*60*2), gc_after_trial=True)
joblib.dump(study, f"NN_hyperparameterstudy{ROLLING_WINDOW_SIZE}.pkl")

Fold 0


100%|██████████| 2399/2399 [00:08<00:00, 284.81it/s]


Epochs: 1 | Train Loss:  0.0185                 | Train F1:  0.6035                 | Val Loss:  0.0139                 | Val F1:  0.7716


100%|██████████| 2399/2399 [00:08<00:00, 273.89it/s]


Epochs: 2 | Train Loss:  0.0107                 | Train F1:  0.8501                 | Val Loss:  0.0137                 | Val F1:  0.7689


...

100%|██████████| 2303/2303 [00:07<00:00, 310.32it/s]


Epochs: 9 | Train Loss:  0.0073                 | Train F1:  0.8956                 | Val Loss:  0.0174                 | Val F1:  0.7286


100%|██████████| 2303/2303 [00:07<00:00, 309.78it/s]
[I 2022-06-16 20:04:02,764] Trial 24 finished with value: 0.7990303301654322 and parameters: {'learning_rate': 3.231584098896318e-05, 'n_layers': 2, 'n_units_l0': 34, 'dropout_l0': 0.25, 'n_units_l1': 26, 'dropout_l1': 0.35000000000000003}. Best is trial 15 with value: 0.807087138352161.


Epochs: 10 | Train Loss:  0.0072                 | Train F1:  0.8971                 | Val Loss:  0.0175                 | Val F1:  0.7284


['NN_hyperparameterstudy10.pkl']

In [25]:
#load
study = joblib.load(f"NN_hyperparameterstudy{ROLLING_WINDOW_SIZE}.pkl")
print("Best trial until now:")
print(" Value: ", study.best_trial.value)
print(" Params: ")
for key, value in study.best_trial.params.items():
    print(f"    {key}: {value}")

Best trial until now:
 Value:  0.807087138352161
 Params: 
    learning_rate: 1.0372272937269755e-05
    n_layers: 2
    n_units_l0: 28
    dropout_l0: 0.45
    n_units_l1: 17
    dropout_l1: 0.4
